#### Bibliotecas

In [1]:
import sys
import tweepy as tw
import pandas as pd

if '/Users/will/Google Drive/workspace/Projetos/projeto_cangaco' not in sys.path:
    sys.path.insert(0, '/Users/will/Google Drive/workspace/Projetos/projeto_cangaco')
if '/Users/will/Google Drive/workspace/Projetos/projeto_cangaco/maria_bonita/conjunto_dados' not in sys.path:
    sys.path.insert(0, '/Users/will/Google Drive/workspace/Projetos/projeto_cangaco/maria_bonita/conjunto_dados')

from utilidades.autenticacao import autenticar
import maria_bonita.conjunto_dados.utilidades.captura.StreamListenerPersonalizado as slp
import maria_bonita.conjunto_dados.utilidades.captura.API_Personalizada as ap
from maria_bonita.conjunto_dados.utilidades.captura.chaves_busca import MISOGINIA
import maria_bonita.conjunto_dados.utilidades.captura.ferramentas_conjunto_dados as fcd
import maria_bonita.conjunto_dados.utilidades.pre_processamento.novas_variaveis as nv

#### Parâmetros

In [2]:
# DEFINIR PARÂMETROS !!!
chaves_escolhidas = [MISOGINIA]

chaves_busca, quantidade_tweets, com_retweets, \
idiomas, caminho_dataset = fcd.parametros(chaves_busca=chaves_escolhidas,
                                          quantidade_tweets=30)

#### Rotina

In [3]:
auth = autenticar()
lista_tweets = []

minha_stream = None
minha_stream = tw.Stream(auth=auth,
                         listener=slp.StreamListenerPersonalizado(lista_resultados=lista_tweets,
                                                                  quantidade_tweets=quantidade_tweets,
                                                                  com_retweets=com_retweets))

minha_stream.filter(languages=idiomas,
                    track=chaves_busca)

1 -----------------------------------------------------------------------
Usuário: juhrockenbach
Texto: nunca aceitem menos do que vocês merecem, é só isso.
------------------------------------------------------------------------

2 -----------------------------------------------------------------------
Usuário: Luzweasley
Texto: @charlieswifee_ so burra esquece 😍
------------------------------------------------------------------------

3 -----------------------------------------------------------------------
Usuário: paixaoopk
Texto: @laurxngerman @NetflixBrasil PUTA QUE PARIU QUE SUSTO
------------------------------------------------------------------------

4 -----------------------------------------------------------------------
Usuário: refugeyoon
Texto: percebendo aqui que eu não aguento nenhum homem sem ser os meus meninos tá puta que pariu
------------------------------------------------------------------------

5 ----------------------------------------------------------------

KeyboardInterrupt: 

#### Visualização

In [4]:
conjunto_dados = pd.DataFrame(lista_tweets)
conjunto_dados.head()

,criado_em,retweet,tweet_id,tweet_url,tweet_texto,tweet_hashtags,tweet_urls_externos,tweet_usuarios_mencionados,tweet_plataforma_origem,tweet_em_resposta,...,usuario_verificado,usuario_localizacao,usuario_descricao,usuario_qtd_seguidores,usuario_qtd_amigos,usuario_qtd_tweets,usuario_url_imagem_perfil,usuario_url_imagem_capa,usuario_criado_em,usuario_json
0,2020-12-08 23:13:59,False,1336448972836261888,https://twitter.com/juhrockenbach/status/13364...,"nunca aceitem menos do que vocês merecem, é só...",[],[],[],iPhone,None,...,False,na casa do homi,onda se tira com títulos || 12.04.20 @NRDubois...,1113,837,38321,https://pbs.twimg.com/profile_images/133606365...,https://pbs.twimg.com/profile_banners/10088623...,2018-06-19 00:01:32,"{'id': 1008862306418221057, 'id_str': '1008862..."
1,2020-12-08 23:13:59,False,1336448973482168321,https://twitter.com/Luzweasley/status/13364489...,@charlieswifee_ so burra esquece §smiling face...,[],[],"[{'screen_name': 'charlieswifee_', 'id_str': '...",iPhone,"{'resposta_ao_tweet': '1336448870474199040', '...",...,False,Slytherin 🐍,(ela/dela) Vidas negras importam§raised fist: ...,1004,1008,7751,https://pbs.twimg.com/profile_images/133427404...,https://pbs.twimg.com/profile_banners/11516536...,2019-07-18 00:43:22,"{'id': 1151653651921981440, 'id_str': '1151653..."
2,2020-12-08 23:14:00,False,1336448974488825856,https://twitter.com/paixaoopk/status/133644897...,@laurxngerman @NetflixBrasil PUTA QUE PARIU QU...,[],[],"[{'screen_name': 'laurxngerman', 'id_str': '12...",Android,"{'resposta_ao_tweet': '1336448711099084800', '...",...,False,Los Angeles; hell,fan account| ...,453,780,5114,https://pbs.twimg.com/profile_images/133644845...,https://pbs.twimg.com/profile_banners/12762742...,2020-06-25 22:01:58,"{'id': 1276274296059580418, 'id_str': '1276274..."
3,2020-12-08 23:14:00,False,1336448974841057280,https://twitter.com/refugeyoon/status/13364489...,percebendo aqui que eu não aguento nenhum home...,[],[],[],Android,None,...,False,ㅤ𝗷𝗶𝗄𝗈𝗼 ‹3 지국. ♥︎%,友 ∷ yo͟ongi ⌕. §guitar§,299,439,1463,https://pbs.twimg.com/profile_images/133565361...,https://pbs.twimg.com/profile_banners/13281985...,2020-11-16 04:49:48,"{'id': 1328198519405076480, 'id_str': '1328198..."
4,2020-12-08 23:14:00,False,1336448974945984517,https://twitter.com/samuelryp/status/133644897...,"ACORDA FILHO DA PUTA, ELA NEM SABE QUE TU EXISTE",[],[],[],Android,"{'resposta_ao_tweet': '1336448843144175617', '...",...,False,rj,20/11 - ele/dele - fã da menina que não sabe q...,348,246,10369,https://pbs.twimg.com/profile_images/133563287...,https://pbs.twimg.com/profile_banners/12788723...,2020-07-03 02:05:20,"{'id': 1278872369524887555, 'id_str': '1278872..."


#### Consolidar dados

In [5]:
if not "conjunto_dados" in locals():
    conjunto_dados = pd.DataFrame(lista_tweets)
conjunto_dados.to_csv(caminho_dataset, sep=';', index=False, encoding='utf-8-sig')

#### Pré-processamento

In [6]:
# Com conexão no end-point

if not "conjunto_dados" in locals():
    conjunto_dados = pd.read_csv(caminho_dataset, sep=';')

API = ap.API()
conjunto_dados['tweet_usuarios_mencionados'] = conjunto_dados.apply(lambda x: fcd.relacionamento_com_mencionados(API=API,
                                                                                                                 usuario_id_origem=x["usuario_id"],
                                                                                                                 mencionados=x['tweet_usuarios_mencionados']),
                                                                    axis=1)

API = ap.API()
conjunto_dados['tweet_em_resposta'] = conjunto_dados.apply(lambda x: fcd.relacionamento_com_respondido(API=API,
                                                                                                       usuario_id_origem=x["usuario_id"],
                                                                                                       mencionados=x['tweet_usuarios_mencionados'],
                                                                                                       em_resposta=x['tweet_em_resposta']), axis=1)

conjunto_dados.to_csv(caminho_dataset, sep=';', index=False, encoding='utf-8-sig')


Chamadas API: 1/179

Chamadas API: 2/179

Chamadas API: 3/179

Chamadas API: 4/179

Chamadas API: 5/179

Chamadas API: 6/179

Chamadas API: 7/179

Chamadas API: 8/179

Chamadas API: 9/179

Chamadas API: 10/179

Chamadas API: 1/170

Chamadas API: 2/170


In [7]:
# Sem conexão no end-point

if not "conjunto_dados" in locals():
    conjunto_dados = pd.read_csv(caminho_dataset, sep=';')
    
conjunto_dados['relacionamento_com_mencionados'] = conjunto_dados['tweet_usuarios_mencionados'].apply(lambda x: nv.relacionamento_com_mencionados(mencionados=x))

conjunto_dados['relacionamento_com_respondido'] = conjunto_dados['tweet_em_resposta'].apply(lambda x: nv.relacionamento_com_respondido(em_resposta=x))

conjunto_dados['texto_original'] = conjunto_dados['tweet_texto']

conjunto_dados['tokens'] = conjunto_dados.apply(lambda x: nv.tokenizar(tweet_texto=x['tweet_texto'],
                                                                       chaves_busca=chaves_escolhidas,
                                                                       mencionados=x['tweet_usuarios_mencionados']),
                                                axis=1)

conjunto_dados['possivel_crime'] = None

conjunto_dados.to_csv(caminho_dataset, sep=';', index=False, encoding='utf-8-sig')